In [2]:
#!/usr/bin/env python
# coding=utf-8

# @file N-Gram.py
# @brief N-Gram
# @author Julia9803,julia925583000@gmail.com
# @version 1.0
# @date 2019-01-11 20:15:27 

import re
from collections import *
import os
import pandas as pd
from math import isnan
N=1
V=0
F=5

basepath = "./subtxn3"
generatepath = "feature/feature3/"
generatelabel = "subtrainLabels3.csv"
contracts=[]

def getPaymentAttribute(filename):
    df = pd.read_csv(filename)
    if len(df[df['direction'] == 'IN']) != 0:
        value_investment = df[df['direction'] == 'IN']['value'].str.split(' ',expand=True)[0].str.replace(',','')
        tmp = pd.DataFrame(value_investment,dtype=float)
        value_investment = tmp.apply(sum).values[0]
        avg_investment = tmp.mean().values[0]
        std_investment = tmp.std().values[0]
        if isnan(std_investment):
            std_investment = 0
    else:
        value_investment = 0.0
        avg_investment = 0.0
        std_investment = 0.0
        
    if len(df[df['direction'] == 'OUT']) != 0:
        value_payment = df[df['direction'] == 'OUT']['value'].str.split(' ',expand=True)[0].str.replace(',','')
        tmp = pd.DataFrame(value_payment,dtype=float)
        value_payment = tmp.apply(sum).values[0]
        avg_payment = tmp.mean().values[0]
        std_payment = tmp.std().values[0]
        if isnan(std_payment):
            std_payment = 0
    else:
        value_payment = 0.0
        avg_payment = 0.0
        std_payment = 0.0
        
    investment_times = len(df[df['direction'] == 'IN'])
    payment_times = len(df[df['direction'] == 'OUT'])
    
    if payment_times != 0:
        investment_payment_rate = investment_times/payment_times
    else:
        investment_payment_rate = investment_times+1 #
        
    txn_period = df['age'].max() - df['age'].min()
    
    
    return value_investment,value_payment,len(df),avg_investment,std_investment,avg_payment,std_payment,investment_times,payment_times,investment_payment_rate,txn_period

## get internal txns' times for directions owner investor other 
## if not internal txn value = 0
def getInternalDirections(filename):
    df = pd.read_csv(filename)
    to_add = filename.split("&")[0]
    from_add = filename.split("&")[1]
    owner = len(df[df['inter_direction'] == 0])
    investor = len(df[df['inter_direction'] == 1])
    other = len(df[df['inter_direction'] == 2])
    return owner,investor,other

def getOpcodeSequence(filename):
    global contracts
    df = pd.read_csv(filename)
#     direction = df['direction'].tolist() 
#     txn_type = df['txn_type'].tolist()
#     _type = df['type'].tolist()
    combine_attribute = df['direction'].str.strip() + '&' + df['txn_type'] + '&' + df['type'].apply(str)
    combine_attribute = combine_attribute.tolist()
    contracts.append({"Id":filename[16:-4],"Class":df['Class'][0]})
    return filename[16:-4],combine_attribute

def getOpcodeNgram(ops, n=N):
    opngramlist = [tuple(ops[i:i+n]) for i in range(len(ops)-n)]
    opngram = Counter(opngramlist)
    return opngram
    
def n_gram():
    map3gram = defaultdict(Counter)
    N_investments = []
    N_payments = []
    D_index = [] # investment-payment
    Txn_len = [] # length of 1to1 txns
    Avg_investment = []
    Std_investment = []
    Avg_payment = []
    Std_payment = []
    Investment_times = []
    Payment_times = []
    Investment_payment_rate = []
    Txn_period = []
    N_owner = []
    N_creator = []
    N_other = []
    I_direction = []
    

    count = 1
    for each_file in os.listdir(basepath):
        print("counting the 3-gram of the {0} file...".format(each_file))
        print(count)
        count += 1
        
        investment,payment,txn_len,avg_investment,std_investment,avg_payment,std_payment,investment_times,payment_times,investment_payment_rate,txn_period = getPaymentAttribute(os.path.join(basepath,each_file))
        N_investments.append(investment)
        N_payments.append(payment)
        D_index.append(investment-payment)
        Txn_len.append(txn_len)
        Avg_investment.append(avg_investment)
        Std_investment.append(std_investment)
        Avg_payment.append(avg_payment)
        Std_payment.append(std_payment)
        Investment_times.append(investment_times)
        Payment_times.append(payment_times)
        Investment_payment_rate.append(investment_payment_rate)
        Txn_period.append(txn_period)
        
        owner,investor,other=getInternalDirections(os.path.join(basepath,each_file))
        N_owner.append(owner)
        N_creator.append(investor)
        N_other.append(other)
        
        ops = getOpcodeSequence(os.path.join(basepath,each_file))
        op3gram = getOpcodeNgram(ops[1])
        map3gram[ops[0]] = op3gram
    
    cc = Counter([])
    for d in map3gram.values():
        cc += d
    selectedfeatures = {}
    tc = 0
    for k,v in cc.items():
        if v >= V:
            selectedfeatures[k] = v
            #  print k,v
            tc += 1
    dataframelist = []
    for fid,op3gram in map3gram.items():
        standard = {}
        standard["Id"] = fid
        for feature in selectedfeatures:
            if feature in op3gram:
                standard[feature] = op3gram[feature]
            else:
                standard[feature] = 0
        dataframelist.append(standard)
        
    df = pd.DataFrame(dataframelist)
    df['Txn_len'] = Txn_len
    df['N_investments'] = N_investments
    df['N_payments'] = N_payments
    df['D_index'] = D_index
    df['Avg_investment'] = Avg_investment
    df['Std_investment'] = Std_investment
    df['Avg_payment'] = Avg_payment
    df['Std_payment'] = Std_payment
    df['Investment_times'] = Investment_times
    df['Payment_times'] = Payment_times
    df['Investment_payment_rate'] = Investment_payment_rate
    df['Txn_period'] = Txn_period
    df['N_owner'] = N_owner
    df['N_creator'] = N_creator
    df['N_other'] = N_other
    
    df.to_csv(generatepath + str(N) + "gramfeature_short.csv",index=False)

    df = pd.DataFrame(contracts)
    df.to_csv(generatelabel,index=False)

if __name__ == '__main__':
    n_gram()

counting the 3-gram of the 0xc5f966a862159eaf0131e1a8f740630a31ea4f3a&0xa3c1e324ca1ce40db73ed6026c4a177f099b5770.csv file...
1
counting the 3-gram of the 0x013871d978d942e072d09ae75a86525278f496e3&0x8d12a197cb00d4747a1fe03395095ce2a5cc6819.csv file...
2
counting the 3-gram of the 0x8b21529cb8c48a2ca7d7cf52c634e87dcdba21ce&0xa3c1e324ca1ce40db73ed6026c4a177f099b5770.csv file...
3
counting the 3-gram of the 0x96858ce64c06e3e0869c8f97b268de2cf72283d0&0x67a5de922c1f4d1a4dbd848466b86b5455fb6a3b.csv file...
4
counting the 3-gram of the 0xb547027a4ccd46ec98199fa88aaedf5aa981db26&0x41a21b264f9ebf6cf571d4543a5b3ab1c6bed98c.csv file...
5
counting the 3-gram of the 0xbb9bc244d798123fde783fcc1c72d3bb8c189413&0x269268008c72694abdf2a372cd6c005c7bd4dfe9.csv file...
6
counting the 3-gram of the 0x867ffb5a3871b500f65bdfafe0136f9667deae06&0x119d0cee798eb5a00ca7afc44edede46994d5267.csv file...
7
counting the 3-gram of the 0x3b15a94016313d44eb9fc6deb0f46b69930b8b55&0xb2816223fddc21aca553ba208c7b7a13b27c207

counting the 3-gram of the 0x8f7ec3ce7abc89c0d001c41e194639d6e25ced33&0x9b5bbb16a7a68660f6503eff6c8e3159820d6396.csv file...
67
counting the 3-gram of the 0x7485ec373c94c862a19df2e176c33fd8c961f7ad&0x6090a6e47849629b7245dfa1ca21d94cd15878ef.csv file...
68
counting the 3-gram of the 0xa74476443119a942de498590fe1f2454d7d4ac0d&0x3b75afda38954180828b588f009ce97b0177e302.csv file...
69
counting the 3-gram of the 0xbb9bc244d798123fde783fcc1c72d3bb8c189413&0x2b0d8470cd14c6ae1f0c4e995644f5bc243adec4.csv file...
70
counting the 3-gram of the 0x765369bc84c3d665ef80e6cb5331b7cb9b7fd4d7&0x6090a6e47849629b7245dfa1ca21d94cd15878ef.csv file...
71
counting the 3-gram of the 0xbefbec5cb64a564d8b5181eb22f6e16eab2b6fea&0x8a883a20940870dc055f2070ac8ec847ed2d9918.csv file...
72
counting the 3-gram of the 0x957c136a3207b979ff79c88b14fb720895e77013&0x8d12a197cb00d4747a1fe03395095ce2a5cc6819.csv file...
73
counting the 3-gram of the 0x4b11819cb5f207ccce6e1b1a09f32863aafd87cd&0x7b307c1f0039f5d38770e15f8043b3dd

counting the 3-gram of the 0xdc7be2718ceb48712aff3a2a27ff7c2d0bdb049c&0x8d12a197cb00d4747a1fe03395095ce2a5cc6819.csv file...
133
counting the 3-gram of the 0x6810e776880c02933d47db1b9fc05908e5386b96&0x81aa1b6e909414f195579b324ce5838f14bd4fdb.csv file...
134
counting the 3-gram of the 0xd0a6e6c54dbc68db5db3a091b171a77407ff7ccf&0xb9e6205be94fc3e4bb8d29517bbf69353132de72.csv file...
135
counting the 3-gram of the 0x98417a14b0791a477ecc9f121eb44d6aabc4899b&0x867ffb5a3871b500f65bdfafe0136f9667deae06.csv file...
136
counting the 3-gram of the 0x78b7fada55a64dd895d8c8c35779dd8b67fa8a05&0x390fbfc706db8538164d0d0e2bf2d9f056e21c84.csv file...
137
counting the 3-gram of the 0x477937962cbcc0419f7a045f43348436726ba749&0xa3c1e324ca1ce40db73ed6026c4a177f099b5770.csv file...
138
counting the 3-gram of the 0x6b6e3700a45df5f035e386a03dde3c29a234721f&0x536a6ba0d913d5d6a4ce2c6eb7ed0de3c0f0b89e.csv file...
139
counting the 3-gram of the 0x514910771af9ca656af840dff83e8264ecf986ca&0xa5f07740506bd2dd632f800b3

counting the 3-gram of the 0x4e8ecf79ade5e2c49b9e30d795517a81e0bf00b8&0x2d9aca11e34e2dd0a1089a3ad0196824c16bdf5b.csv file...
200
counting the 3-gram of the 0xa26e44efd06b7ef98cd6517b27f7eedd781766fd&0x7b307c1f0039f5d38770e15f8043b3dd26da5e8f.csv file...
201
counting the 3-gram of the 0x96a65609a7b84e8842732deb08f56c3e21ac6f8a&0xf3f2f2053dd55b90adef5429136ad2ce9c7f7bee.csv file...
202
counting the 3-gram of the 0xbb5f89ce4c615b5f69523a740df73d80872dbdf1&0x536a6ba0d913d5d6a4ce2c6eb7ed0de3c0f0b89e.csv file...
203
counting the 3-gram of the 0xbf1027944e14c417b299b3c0a6a8dd7a294ef9c7&0xfde889c9354d09cec72845e76a0d9f97f4686f7a.csv file...
204
counting the 3-gram of the 0x4e8ecf79ade5e2c49b9e30d795517a81e0bf00b8&0x6a77ca15715bb32f087c277d7a681a0f8baff927.csv file...
205
counting the 3-gram of the 0xd010d7765b51280ddbeb1c2cdb7028d4751de34a&0x4e8ecf79ade5e2c49b9e30d795517a81e0bf00b8.csv file...
206
counting the 3-gram of the 0x4e8ecf79ade5e2c49b9e30d795517a81e0bf00b8&0x210361dab172c028386fc18a4

counting the 3-gram of the 0x0389a06b028526b05966c287370bebefa0082176&0x1bd563eda662e0667868a5f692b60bb68acc8888.csv file...
268
counting the 3-gram of the 0x861ab771d6787ac7eecc2d33b6fed63a06ae7d1c&0xac4839e46d66dc7485b937cd1a4f6eec15043d96.csv file...
269
counting the 3-gram of the 0xaf4dce16da2877f8c9e00544c93b62ac40631f16&0xedb5f8ac086204c932c6292f0ac8d38de250f57b.csv file...
270
counting the 3-gram of the 0x876f2a2526c124fdc064b1401e72057db093421d&0x37afb28c63a9b9a581a91532355e1ec66ffd2239.csv file...
271
counting the 3-gram of the 0x51a3a664dadf491740c6d04b7591db0c5d839ea7&0x4e8ecf79ade5e2c49b9e30d795517a81e0bf00b8.csv file...
272
counting the 3-gram of the 0x21eb3ea85b425bb4fd85737a7cd9276df0609349&0xaaab2ec23dd5dd9602e631b8399fa94c9d134b3a.csv file...
273
counting the 3-gram of the 0x5eaac0df2ac25e2d310203a89499721c80d6f91c&0x4e8ecf79ade5e2c49b9e30d795517a81e0bf00b8.csv file...
274
counting the 3-gram of the 0x16f0a248c359a4f98b5b0284149a7dba73b45030&0x536a6ba0d913d5d6a4ce2c6eb

...

counting the 3-gram of the 0xd34abc43f6260f9317abdaa229156c745c7565d1&0x6090a6e47849629b7245dfa1ca21d94cd15878ef.csv file...
19697
counting the 3-gram of the 0x06f738cbb570e2a03d2a65353072b6140f3b86ef&0x6090a6e47849629b7245dfa1ca21d94cd15878ef.csv file...
19698
counting the 3-gram of the 0xf2a50b8fb5c58df22a42d68bc16d6c730973cc88&0x8a883a20940870dc055f2070ac8ec847ed2d9918.csv file...
19699
counting the 3-gram of the 0xa683c1b815997a7fa38f6178c84675fc4c79ac2b&0xb547027a4ccd46ec98199fa88aaedf5aa981db26.csv file...
19700
counting the 3-gram of the 0x4b940a2610ce87e005f9a883588f3d9d5d930002&0xa3c1e324ca1ce40db73ed6026c4a177f099b5770.csv file...
19701
counting the 3-gram of the 0xa92073cfadb8c1e12450ba01b329bde763dec8b9&0xb1690c08e213a35ed9bab7b318de14420fb57d8c.csv file...
19702
counting the 3-gram of the 0xf230b790e05390fc8295f4d3f60332c93bed42e2&0x1d3f73f5ff9d17e84875f254546ba6bdccb9f858.csv file...
19703
counting the 3-gram of the 0x4e8ecf79ade5e2c49b9e30d795517a81e0bf00b8&0xbfef7e251af

counting the 3-gram of the 0x12faa19c6d3deece7c1eee2b9661c615e7530372&0x755cdba6ae4f479f7164792b318b2a06c759833b.csv file...
19770
counting the 3-gram of the 0xb2b1fd7aae1ec069d594ceea3fd4d6fc5bc23ceb&0x4e8ecf79ade5e2c49b9e30d795517a81e0bf00b8.csv file...
19771
counting the 3-gram of the 0x04313aba007d08427ab2cf1bc7df00d0cd74d40b&0xa3c1e324ca1ce40db73ed6026c4a177f099b5770.csv file...
19772
counting the 3-gram of the 0xc61ed74017d66eceb5eee1f20a012e4774cd79f0&0xaaab2ec23dd5dd9602e631b8399fa94c9d134b3a.csv file...
19773
counting the 3-gram of the 0xb2f7eb1f2c37645be61d73953035360e768d81e6&0x338e25a8fa1b3f55cdb3877e9d0fed1f00fa8033.csv file...
19774
counting the 3-gram of the 0xd850942ef8811f2a866692a623011bde52a462c1&0xb6855a1ba0f6e5acc51cc9c57554ba7e6642cc8b.csv file...
19775
counting the 3-gram of the 0x27054b13b1b798b345b591a4d22e6562d47ea75a&0x03fe31663dba7f8200482d39185daf2be283a293.csv file...
19776
counting the 3-gram of the 0xd549f37d0b5b5434994c477f2a85fdad96f6da8e&0x755cdba6ae4

counting the 3-gram of the 0x1a79e687938454b8e8a518a321f53288f68816ae&0xb2816223fddc21aca553ba208c7b7a13b27c207c.csv file...
19840
counting the 3-gram of the 0x33b966b47b6586d588939555d4b5cfb769128911&0xb10ea0d397db942a1083a4af9c6074ab63bc8c8b.csv file...
19841
counting the 3-gram of the 0xd4ac200707055eb4580009bed1a2190566780324&0x4e8ecf79ade5e2c49b9e30d795517a81e0bf00b8.csv file...
19842
counting the 3-gram of the 0x8aa33a7899fcc8ea5fbe6a608a109c3893a1b8b2&0x5b9b5abc0df852e4fb3bb63e59c5925cd8791e48.csv file...
19843
counting the 3-gram of the 0x2c4e8f2d746113d0696ce89b35f0d8bf88e0aeca&0x033911605d48e1293469570f46eb64acb3a81b8b.csv file...
19844
counting the 3-gram of the 0xed4129c8a692309e185ac1e1858200c2a0aa8e65&0x8d12a197cb00d4747a1fe03395095ce2a5cc6819.csv file...
19845
counting the 3-gram of the 0x6b754fba448bce98000b7ea060b0771074d1123b&0x6090a6e47849629b7245dfa1ca21d94cd15878ef.csv file...
19846
counting the 3-gram of the 0x050394cb5ee3019dcd1516296172d7b69f8a28f5&0xc7af99fe551

counting the 3-gram of the 0x04bd24d829f88ff3d7423be710744e3e0764660d&0x7b307c1f0039f5d38770e15f8043b3dd26da5e8f.csv file...
19912
counting the 3-gram of the 0xad565956ae5bd43117f6b0a650ec18c621ff8e0d&0x8036165ccabee251c886d6121b047c16b5e798b5.csv file...
19913
counting the 3-gram of the 0x4e8ecf79ade5e2c49b9e30d795517a81e0bf00b8&0xbbc818f10b2bd5d4ae1a8e3717f6324acee6ec1f.csv file...
19914
counting the 3-gram of the 0xf859b5a00c9fe44b08b05283b1490bb4d98db691&0x459f7854776ed005b6ec63a88f834fdab0b6993e.csv file...
19915
counting the 3-gram of the 0x5462a53d1d95594ef8a99fb45634671cea06c858&0xb547027a4ccd46ec98199fa88aaedf5aa981db26.csv file...
19916
counting the 3-gram of the 0x42d6622dece394b54999fbd73d108123806f6a18&0xf5dde0f87da26dbd7e5166c306ff780bf7d16aa2.csv file...
19917
counting the 3-gram of the 0x7692a4436f8252f34258c2bfa846ec5e840d5e55&0x4e8ecf79ade5e2c49b9e30d795517a81e0bf00b8.csv file...
19918
counting the 3-gram of the 0xd0a6e6c54dbc68db5db3a091b171a77407ff7ccf&0x85b1acae7a9

counting the 3-gram of the 0xfffffff1ac8dcf49f4afd3f90995120e568740f9&0x4e8ecf79ade5e2c49b9e30d795517a81e0bf00b8.csv file...
19983
counting the 3-gram of the 0xde99e7c5c664aecc3e3427cf7aa4a6d600ed39da&0x6090a6e47849629b7245dfa1ca21d94cd15878ef.csv file...
19984
counting the 3-gram of the 0xfc3f77e6f6728962561487e28d287dac276aa4de&0x867ffb5a3871b500f65bdfafe0136f9667deae06.csv file...
19985
counting the 3-gram of the 0x3adb2f652e9c835fc2860d326add5fa955004940&0x4e8ecf79ade5e2c49b9e30d795517a81e0bf00b8.csv file...
19986
counting the 3-gram of the 0x57e2a03bbbbb7636e0c6a40a654147ea7d89c47e&0x6090a6e47849629b7245dfa1ca21d94cd15878ef.csv file...
19987
counting the 3-gram of the 0x2206c2bb68b7ee9521e19115c00394f086c9ddab&0x536a6ba0d913d5d6a4ce2c6eb7ed0de3c0f0b89e.csv file...
19988
counting the 3-gram of the 0x5adc88216d35c3aef3a9192e26b84c50667eb6e1&0x702392282255f8c0993dbbbb148d80d2ef6795b1.csv file...
19989
counting the 3-gram of the 0x89d24a6b4ccb1b6faa2625fe562bdd9a23260359&0xf9d8bd1c6b9